In [1]:
# Data from 
# https://smartgrowthamerica.org/wp-content/uploads/2023/01/Foot-Traffic-Ahead-2023.1.pdf
# https://www.bls.gov/web/metro/laummtrk.htm

#import libraries
import pandas as pd

#set view options
pd.set_option('display.max_columns', None)

In [2]:
#import data
# cpi = pd.read_excel("data_raw/statistic_id245014_us-consumer-price-index-for-selected-us-cities-2022.xlsx")
pp = pd.read_csv("data_raw/Purchasing Power Map Most Expensive US Cities  Metros (1).csv")
walk = pd.read_csv("data_raw/Foot  Traffic  AHEAD - Sheet1.csv")
unemploy = pd.read_csv("data_raw/Unemployment - Sheet1.csv")

In [3]:
#drop unwanted cols
#walk
column_names = [
    "RANK",
    "OFFICE SHARE",
    "MULTIFAMILY RENTAL SHARE",
    "RETAIL SHARE",
    "FOR SALE SHARE",
    "COMBINED SHARE"
]
walk.drop(columns=column_names, inplace=True)
#employ
unemploy.drop(columns=["Rank"], inplace=True)
#sep cols
unemploy[['Metropolitan', 'Statistical Area']] = unemploy['Metropolitan area'].str.split(',', 1, expand=True)
#keep rows for merge 
keep_metro = [
    "New York-Newark-Jersey City", "Boston-Cambridge-Nashua", "Washington-Arlington-Alexandria", "Seattle-Tacoma-Bellevue", "Portland-Vancouver-Hillsboro", "San Francisco-Oakland-Hayward", "Chicago-Naperville-Elgin", "Los Angeles-Long Beach-Anaheim", "Pittsburgh", "Philadelphia-Camden-Wilmington", "Minneapolis-St. Paul-Bloomington", "Miami-Fort Lauderdale-West Palm Beach", "Charlotte-Concord-Gastonia", "Austin-Round Rock", "Atlanta-Sandy Springs-Roswell", "Denver-Aurora-Lakewood", "Cleveland-Elyria", "Houston-The Woodlands-Sugar Land", "Columbus", "Baltimore-Columbia-Towson", "Kansas City", "Nashville-Davidson--Murfreesboro--Franklin", "St. Louis", "Sacramento--Roseville--Arden-Arcade", "Cincinnati", "Detroit-Warren-Dearborn", "Dallas-Fort Worth-Arlington", "San Diego-Carlsbad", "Indianapolis-Carmel-Anderson", "Tampa-St. Petersburg-Clearwater", "Virginia Beach-Norfolk-Newport News", "Phoenix-Mesa-Scottsdale", "Orlando-Kissimmee-Sanford", "San Antonio-New Braunfels", "Las Vegas-Henderson-Paradise"
]
unemploy = unemploy[unemploy['Metropolitan'].isin(keep_metro)]
#replace names for merge
change_metro = {
    "New York-Newark-Jersey City":"New York",
    "Boston-Cambridge-Nashua":"Boston",
    "Washington-Arlington-Alexandria":"Washington, DC",
    "Seattle-Tacoma-Bellevue":"Seattle",
    "Portland-Vancouver-Hillsboro": "Portland",
    "San Francisco-Oakland-Hayward":"San Francisco",
    "Chicago-Naperville-Elgin":"Chicago",
    "Los Angeles-Long Beach-Anaheim":"Los Angeles",
    "Pittsburgh":"Pittsburgh",
    "Philadelphia-Camden-Wilmington":"Philadelphia",
    "Minneapolis-St. Paul-Bloomington":"Minneapolis-St. Paul",
    "Miami-Fort Lauderdale-West Palm Beach":"Miami",
    "Charlotte-Concord-Gastonia":"Charlotte",
    "Austin-Round Rock":"Austin",
    "Atlanta-Sandy Springs-Roswell":"Atlanta",
    "Denver-Aurora-Lakewood":"Denver",
    "Cleveland-Elyria":"Cleveland",
    "Houston-The Woodlands-Sugar Land":"Houston",
    "Columbus":"Columbus",
    "Baltimore-Columbia-Towson":"Baltimore",
    "Kansas City":"Kansas City",
    "Nashville-Davidson--Murfreesboro--Franklin":"Nashville",
    "St. Louis":"St. Louis",
    "Sacramento--Roseville--Arden-Arcade":"Sacramento",
    "Cincinnati":"Cincinnati",
    "Detroit-Warren-Dearborn":"Detroit",
    "Dallas-Fort Worth-Arlington":"Dallas-Fort Worth",
    "San Diego-Carlsbad":"San Diego",
    "Indianapolis-Carmel-Anderson":"Indianapolis",
    "Tampa-St. Petersburg-Clearwater":"Tampa",
    "Virginia Beach-Norfolk-Newport News":"Virginia Beach",
    "Phoenix-Mesa-Scottsdale":"Phoenix",
    "Orlando-Kissimmee-Sanford":"Orlando",
    "San Antonio-New Braunfels":"San Antonio",
    "Las Vegas-Henderson-Paradise":"Las Vegas"
}
unemploy['Metropolitan'] = unemploy['Metropolitan'].replace(change_metro)
#drop cols from unemploy
column_names = [
   "Metropolitan area", "Statistical Area"
]
unemploy.drop(columns=column_names, inplace=True)
#split col
pp[['REGION', 'other']] = pp['MSA'].str.split(',', 1, expand=True)
#drop cols from unemploy
column_names = [
   "MSA", "County", "State", "other"
]
pp.drop(columns=column_names, inplace=True)
#drop duplicate rpws
pp = pp.drop_duplicates()
#keep rows like
desired_cities = ['New York', 'Boston', 'Washington, DC', 'Seattle', 'Portland', 'San Francisco', 'Chicago', 'Los Angeles', 'Pittsburgh', 'Philadelphia', 'Minneapolis-St. Paul', 'Miami', 'Charlotte', 'Austin', 'Atlanta', 'Denver', 'Cleveland', 'Houston', 'Columbus', 'Baltimore', 'Kansas City', 'Nashville', 'St. Louis', 'Sacramento', 'Cincinnati', 'Detroit', 'Dallas-Fort Worth', 'San Diego', 'Indianapolis', 'Tampa', 'Virginia Beach', 'Phoenix', 'Orlando', 'San Antonio', 'Las Vegas', 'WT. AVG']
pattern = '|'.join(desired_cities)
pp = pp[pp['REGION'].str.contains(pattern)]
#convert text for merge
pp['REGION'] = pp['REGION'].replace(change_metro)
#convert text for merge
change_metro2 = {
    "San Francisco-Oakland-Berkeley":"San Francisco",
    "San Diego-Chula Vista-Carlsbad":"San Diego",
    "Miami-Fort Lauderdale-Pompano Beach":"Miami",
    "Boston-Cambridge-Newton":"Boston",
    "Sacramento-Roseville-Folsom":"Sacramento",
    "Portland-South Portland":"Portland",
    "Austin-Round Rock-Georgetown":"Austin",
    "Phoenix-Mesa-Chandler":"Phoenix",
    "Atlanta-Sandy Springs-Alpharetta":"Atlanta"
}
pp['REGION'] = pp['REGION'].replace(change_metro2)

/tmp/ipykernel_489614/1295086551.py:15: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  unemploy[['Metropolitan', 'Statistical Area']] = unemploy['Metropolitan area'].str.split(',', 1, expand=True)
/tmp/ipykernel_489614/1295086551.py:66: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  pp[['REGION', 'other']] = pp['MSA'].str.split(',', 1, expand=True)


In [4]:
#merge unemployment and walkability
merged_df = pd.merge(walk, unemploy, how='left', left_on='REGION', right_on="Metropolitan")
# remove columbus
merged_df = merged_df[merged_df['REGION'] != 'Columbus']
merged_df = merged_df[merged_df['REGION'] != 'WT. AVG']
#merge in purchase power
merged_df = pd.merge(merged_df, pp, how='left', on='REGION')
#drop unwanted col
merged_df.drop(columns=['Metropolitan'], inplace=True)
#renam cols
merged_df.rename(columns={'FOOT TRAFFIC AHEAD INDEX': 'Walkable', "February 2024(P)\nrate":"Unemployment", "$100 Value": "Purchase_power_100_dollars"}, inplace=True)
merged_df['Purchase_power_100_dollars'] = merged_df['Purchase_power_100_dollars'].replace('[\$,]', '', regex=True).astype(float)

#replace value deleted by mistake
merged_df.loc[merged_df['REGION'] == 'Washington, DC', 'Purchase_power_100_dollars'] = 89.82

In [5]:
merged_df.head(40)


,REGION,Walkable,Unemployment,Purchase_power_100_dollars
0,New York,100.0,4.7,87.28
1,Boston,74.3,3.3,91.17
2,"Washington, DC",72.6,3.0,89.82
3,Seattle,69.4,4.7,87.28
4,Portland,68.5,4.8,94.90
5,Portland,68.5,4.8,98.29
6,San Francisco,66.2,4.4,83.45
7,Chicago,65.9,5.3,94.86
8,Los Angeles,59.0,4.8,87.86
9,Pittsburgh,57.2,3.6,105.09


In [7]:
merged_df.to_csv('./data_partial/walk_pp_unemploy.csv', index=False)